# Finding the Best Transformation
In the previous section, we established that we can register two images together by creating a mapping

$$
\mathbf{Q} = (\mathbf{TM}_{S})^{-1}\mathbf{M}_{T}
$$

where $\mathbf{T}$ is some affine transformation that renders the anatomical alignment between the images as good as possible. We could approach finding the best $\mathbf{T}$ via manual registration of the images, but this would soon get impractical for large numbers of subjects. As such, not only are we looking for the best $\mathbf{T}$ we can find, but we are also looking for a way to find it *automatically* using a computer. 

In order to do this, we are going to need some way of quantifying what we mean by *best*. This means we need some numeric value that tells us how good the alignment between the images is. We can then try different transformations in $\mathbf{T}$, recalculate this value and see if the registration has improved or not. In theory, we could just keep doing this until no changes we make to $\mathbf{T}$ makes the alignment any better. At this point, we assume we have found the best $\mathbf{T}$ and stop.

## Objective functions
In order to measure how well-aligned two images are, we make use of an *objective function*. This is an equation that uses the voxel values from both images to calculate a number that tells us how well-registered the images are. Depending on the function, our aim is to find a $\mathbf{T}$ that makes this number as *big* or as *small* as possible. Most imaging software provides a choice of several different objectives, each of which has their uses and limitations. We will go through some of the main ones available in `SPM` below.

### Least-squares
One of the most basic objective functions we can use is least-squares. This is based on adding up the differences between the voxel values of the two images. These differences are squared to prevent positive and negative differences cancelling out in the sum, as formalised in the equation below 

$$
c = \sum_{v=1}^{n}(T_{v} - S_{v})^{2}
$$

where $T_v$ and $S_v$ indicate the value of voxel $v$ from the *target* and *source* images. Given that the sum is over $n$ voxels, it is assumed that both images are the same dimensions. If not, the source needs to be interpolated to the same dimensions as the target. The idea is that images from the same modality will have similar voxel values when they are well-aligned, compared to when they are not. As such, there will be bigger differences in voxel values when the images are misaligned and thus the sum of squared differences will also be bigger. Therefore, the *smaller* you can make the sum of squared differences, the better the alignment should be.

One big advantage of the least-squares function is that it is very fast to calculate. However, a disadvantage is that it is only suitable for *within-modality* registration. As such, if we want to register two BOLD fMRI images, or two T1-weighted images, this approach will work well. However, if we wanted to register an fMRI image to a T1 image, then this cost function will perform poorly. In `SPM`, least-squares is mainly used for fast motion correction and is not available as an option elsewhere.

### Information Theory
In the early days of image registration, cost functions tended to be simple metrics such as least-squares. However, it was soon discovered that more flexible cost functions could be derived using the principles of *information theory*. Although the full foundations of information theory are beyond the scope of this lesson (see [Stone (2015)](https://www.librarysearch.manchester.ac.uk/permalink/44MAN_INST/bofker/alma992975877309801631) if you want to learn more), the aim here is to try and give a flavour of the theory so you can see why it is helpful for image registration problems. 

Within information theory there is a key concept known as *entropy*, which measures randomness or uncertainty. When applied to an image, entropy is similar in spirit to variance, as the greater the  variety of intensities, the larger the entropy becomes. This is because the image gets less predictable. As such, entropy measures where an image is on the scale of being perfectly *predictable* to completely *unpredictable*. This can be seen in {numref}`entropy-fig`, where $H(A)$ indicates the entropy of a perfectly predictable image (where all voxel values are the same), a brain image and an image of random noise. Below the images are histograms of the intensity values in the image. These are used in the calculation of entropy as they reflect the probability of different values occurring in the image. The more *structure* there is in the histogram, the more predictable the image and the lower the entropy. 

```{figure} images/entropy.png
---
width: 800px
name: entropy-fig
---
Illustration of entropy for different images.
```

When we have multiple images we can calculate a value known as *joint entropy*. This indicates the amount of uncertainty when we consider both images together. The more that one image is able to predict the other, the smaller this value becomes. If the two images were identical then we could perfectly predict one from the other and the joint entropy would be 0. As such, the more misaligned the images become, the more the joint entropy would increase. As we saw above, this concept of *predictability* is tied to the concept of a histogram. When dealing with joint entropy, we use a *joint histogram*, where the values indicate the co-occurrance of certain combinations of intensities across the two images. For example, {numref}`joint-hist-fig` shows joint histograms from two images after applying rotations that make the alignment worse. The colours within the histogram reflect the height of the bars. As such, you can think of a joint histogram like viewing a mountain range from the top-down, where the heights of the mountain are indicated by how bright the colour is. As we can see, there is less structure and the histogram becomes more "smeary". 

```{figure} images/joint-hist.png
---
width: 800px
name: joint-hist-fig
---
Illustration of joint-histograms for two images through different degrees of rotation that makes the alignment worse.
```

```{Important}
The most important point about joint-entropy is that it is calculated using the *probabilities* expressed in the joint-histograms, rather than directly comparing the intensity values of the images. As such, it can be used for both within-modality *and* between-modality registration problems.
```

At this point, it may seem reasonable to use joint-entropy as the objective function. Although there have been papers using joint-entropy in the past (see [Jenkinson & Smith (2001)](https://pubmed.ncbi.nlm.nih.gov/11516708/) for some discussion), `SPM` instead chooses to use a closely related value called *mutual information*. The main thing to note is that mutual information reflects the amount of information that you can learn about Image B by first observing Image A. In other words, it tells us how much uncertainty about Image B is *reduced* after observing Image A. If A and B are completely unrelated then knowing something about A tells us nothing about B and the mutual information is 0. Alternatively, if A and B are the same image then observing A tells us everything about B and the mutual information is at its maximum. Given that we can learn more about B if A and B are aligned, mutual information will increase as registration improves. Mutual information therefore provides an alternative objective function, noting that a *normalised* form of mutual information is often used in practise as evidence suggests that it performs better in a wider variety of cases and is less sensitive to the degree of image overlap (see [Studholme *et al.*, 1999](https://www.sciencedirect.com/science/article/abs/pii/S0031320398000910)). 

## Optimisation
Once we have defined a suitable objective function, we need some way of searching through many possible values for $\mathbf{T}$ to see which one produces the smallest or largest function value. This is where we get into the world of optimisation algorithms. Understanding how these forms of algorithm work in any sort of detail is far beyond the scope of this course. As such, the main thing to know is that these algorithms have a principled way of searching through lots and lots of possibilities for our transformation $\mathbf{T}$. They use the result of the objective function to make decisions about how to change $\mathbf{T}$ to try and make the value smaller or bigger. Once the algorithm thinks it cannot make tany improvements to the objective function value, it stops. 

To understand this a little more, imagine that we could only change two transformation parameters. For the sake of argument, let us make these the millimetre translations along the $x$ and $y$ axes. The situation faced by optimisation is that across all the possible values of these translations, there is only one combination that will give us the the desired value of the objective function. In the case of *minimisation* the objective function is usually referred to as a *cost function*. The smallest possible value of the cost function (called the *global minimum*) is potentially surrounded by many other peaks and troughs that the algorithm has to navigate. This can be seen visually in {numref}`param-space-fig`. 

```{figure} images/param-space.png
---
width: 800px
name: param-space-fig
---
Illustration of the concept of a parameter space, with the global minimum and a local minimum highlighted.
```
In this situation, what makes finding the global minimum so difficult is that the algorithm cannot see the landscape of possible values. As such, it must explore blind, building up a picture of the mountains and valleys little-by-little. What we need is for the algorithm to be able to explore this landscape thoroughly enough that it can deduce where the deepest valley is, but without taking an impractical amount of time to do so.

Despite the complexity of this situation, there are many algorithms that can be used to try and find the global minimum or maximum within an unknown landscape of values. For our purposes, one of the more important elements of these algorithms to understand is how they choose to stop exploring. This is usually based on the heuristic that if the algorithm reaches a point where no matter which direction it goes the cost function increases, it will assume it has found the deepest valley and will stop. The issue here, of course, is that it may have found a valley, but that valley may not be the deepest. We say that the algorithm has got stuck in a *local minimum*. This is clearly problematic because the algorithm will finish on a solution that is not the best possible solution. In fact, it may not be a very good solution at all. 

The only way we can prevent this from happening is by starting the algorithm off in a position as close to the optimal solution as possible. For imaging, this means roughly aligning the images manually first and then letting the optimisation algorithm polish off the alignment as much as it wants.

```{important}
For image registration, the easiest way to provide good starting values for optimisation is to set the origin of each image to roughly the same anatomical location. The origin corresponds to the location in the image that has associated millimetre coordinates of $[0, 0, 0]$. This point represents the *start* of the millimetre coordinate system where all the axes cross. This is particularly important because images are aligned in world-space by implicitly matching their origins. As such, if one image has the origin located in the frontal lobe and the other image has the origin located in the cerebellum, they are never going to be aligned anatomically. This was precisely the problem with the example images at the very beginning of this lesson.
```